<a href="https://colab.research.google.com/github/uzma-knpc/Generative-AI/blob/main/RAG-AEMCK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG ABOUT CANCER TREATEMNT AT AEMCK

In [70]:
!pip install langchain-google-genai

!pip install -q -U google-generativeai

In [71]:
import google.generativeai as genai
from google.colab import userdata

genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))

In [73]:
from typing import Dict #typing module provides type hints

result : Dict = genai.embed_content(
    model="models/text-embedding-004",
    content="What is the meaning of life?",
    task_type="retrieval_document",
    title="Embedding of single string",
)

# # 1 input > 1 vector output
#print(str(result["embedding"])[:50], "... TRIMMED]")
#result['embedding']

In [75]:
from typing import Dict

result : Dict = genai.embed_content(
    model="models/text-embedding-004",
    content=[
        "What is the meaning of life?",
        "How much wood would a woodchuck chuck?",
        "How does the brain work?",
    ],
    task_type="retrieval_document",
    title="Embedding of list of strings",
)

# A list of inputs > A list of vectors output
#for v in result["embedding"]:
    #print(str(v)[:50], "... TRIMMED ...", len(v))

# Building Vector Stores & Retreival using Chroma DB

In [76]:
!pip install -Uq langchain-chroma

In [11]:
import getpass
import os

In [104]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="•	Cancer is a group of diseases characterized by the uncontrolled growth and spread of abnormal cells. These cells can invade and damage surrounding tissues and organs. They can originate in any part of the body and, if not detected and treated early, can be life-threatening.",
        metadata={"source": "aemck-cancer-doc"},
    ),
    Document(
        page_content="•	aemck is Atomic energy cancer hospital, which is pioneer hospital of Pakistan atomic energy commission PAEC . estableish on 1971.",
        metadata={"source": "aemck-cancer-doc"},
    ),
    Document(
        page_content="Early detection is vital for successful treatment. Screening tests and examinations can help in early identification",
        metadata={"source": "aemck-cancer-earlydetection"},
    ),
    Document(
        page_content="Treatment options vary depending on the type and stage of cancer and may include Surgery: To remove tumors	Chemotherapy: Using drugs to kill cancer cells•	Radiation Therapy: Using high-energy rays to destroy cancercells•	Immunotherapy: Boosting the body's immune system to fight cancer cells•	Targeted Therapy: Drugs targeting specific abnormalities within cancer cells•	Hormone therapy: Blocks cancer cells from getting the hormones they need to grow.",
        metadata={"source": "aemck-cancer-treatment"},
    ),
    Document(
        page_content="•	Breast cancer is a type of cancer that forms in the cells of the breast. It can occur in both men and women, but it's far more common in women. It usually begins in the milk-producing glands (lobules).",
        metadata={"source": "aemck-cancer-cancerinwomen"},
    ),
    Document(
        page_content="•	Breast Self-Exams (BSE): Women can perform monthly self-exams to detect any changes in their breasts.•	Clinical Breast Exam (CBE): Conducted by a healthcare professional to check for lumps or other abnormalities.Mammography: A crucial tool for breast cancer detection.•	Regular mammograms and prompt medical attention if any changes are noticed in the breasts are crucial for early detection and treatment. Early-stage detection significantly improves the chances of successful treatment and survival.",

        metadata={"source": "aemck-cancer-diagnosis"},
    ),
    Document(
        page_content="•	Preventive measures for oral cancer include:Avoiding Tobacco: Both smoking and smokeless tobacco increase the risk.Limiting Alcohol Consumption: Excessive alcohol intake is a risk factor.Good Oral Hygiene: Regular dental check-ups and maintaining oral health.",
        metadata={"source": "aemck-cancer-preventive"},
    ),
    Document(
        page_content=" Oral cancer refers to any cancer that develops in the mouth, including the lips, tongue, gums, floor of the mouth, and roof of the mouth. It typically manifests as a sore or growth that doesn't go away. Risk factors include tobacco use, heavy alcohol consumption, HPV infection, and poor oral hygiene.",
        metadata={"source": "aemck-cancer-cancerinment"},
    ),

]
print(f"document[0]{documents[1].page_content}is")

document[0]•	aemck is Atomic energy cancer hospital, which is pioneer hospital of Pakistan atomic energy commission PAEC . estableish on 1971.is


In [105]:
!pip install -Uq langchain-google-genai

In [106]:
!pip install  langchain-community

import google.generativeai as genai
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings


embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",
                                          google_api_key=userdata.get('GOOGLE_API_KEY'))
# embeddings.embed_query("What's our Q1 revenue?")

In [107]:
from langchain_chroma import Chroma
# from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(
    documents,
    embedding=embeddings)

In [108]:
embedding = embeddings.embed_query("aemck")# convert cat into vector

vectorstore.similarity_search_by_vector(embedding)

[Document(metadata={'source': 'aemck-cancer-doc'}, page_content='•\taemck is Atomic energy cancer hospital, which is pioneer hospital of Pakistan atomic energy commission PAEC . estableish on 1971.'),
 Document(metadata={'source': 'aemck-cancer-earlydetection'}, page_content='Early detection is vital for successful treatment. Screening tests and examinations can help in early identification'),
 Document(metadata={'source': 'aemck-cancer-earlydetection'}, page_content='Early detection is vital for successful treatment. Screening tests and examinations can help in early identification'),
 Document(metadata={'source': 'aemck-cancer-diagnosis'}, page_content='•\tBreast Self-Exams (BSE): Women can perform monthly self-exams to detect any changes in their breasts.•\tClinical Breast Exam (CBE): Conducted by a healthcare professional to check for lumps or other abnormalities.Mammography: A crucial tool for breast cancer detection.•\tRegular mammograms and prompt medical attention if any change

# Retrievers

In [164]:
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=4)  # select top result

#retriever.batch(["cancer"])

In [165]:
!pip install -Uq langchain-google-genai

In [166]:
from langchain_google_genai import ChatGoogleGenerativeAI



llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",
                             api_key = userdata.get('GOOGLE_API_KEY')
)

In [ ]:
#NOT USED
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context and prompt .

{question}

Context:
{context}
"""

In [167]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Determine the source of information for the following question.

Question:
{question}


If the question is related to the provided document, use the context below to answer.
Otherwise, extract the answer from the internet.

Context from Document and LLM:
{context}

Final Response:
"""


In [128]:
#NOT USED
def create_chat_message(question, context):
    return f"""
    Answer this question using the provided context and prompt.

    Question:
    {question}

    Context:
    {context}
    """

# Use the function
question = "What is PAEC?"
context = "PAEC stand for PAKISTAN ATOMIC ENERGY COMMISSION."

message = create_chat_message(question, context)
print(message)



    Answer this question using the provided context and prompt.

    Question:
    <class 'langchain_core.runnables.passthrough.RunnablePassthrough'>

    Context:
    bound=RunnableLambda(similarity_search) kwargs={'k': 1} config={} config_factories=[]
    


In [171]:
prompt = ChatPromptTemplate.from_messages([("ai",message)])


In [180]:
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm
#rag_chain={"COMPLETION": retriever,"Prompt": RunnablePassthrough()}|prompt|llm



In [182]:
query1="what is the most common cancer in women"
response = rag_chain.invoke(query1)

print(f"\033[1;32m Agent Query:{query1}.\033[0m")
print(f"\033[44;43m Answer:{response.content}.\033[0m ")


 Agent Query:what is the most common cancer in women.
 Answer:Based on the provided text, breast cancer is the most common cancer in women.  The source of this information is the multiple documents with the source 'aemck-cancer-cancerinwomen'.. 
